In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
filenum = 2
expid = 1

In [ ]:
train_df = pd.read_csv("fashion-mnist_train.csv")
test_df = pd.read_csv("fashion-mnist_test.csv")
print(train_df.shape , test_df.shape)
print("Train Cols : " , train_df.columns)
print("Test Cols : " , test_df.columns)
print(" Label -  " , [i for i in train_df.columns if i not in test_df.columns] )

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor , Lambda

class CustomMNISTDataset(Dataset):
    def __init__(self, csv_name, img_dir, transform=None, target_transform=None , label_name = "label"):
        
        self.img_filename = csv_name
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.label_name = label_name
        
        img_path = os.path.join(self.img_dir, self.img_filename)
        self.img_df = pd.read_csv(img_path)

    def __len__(self):
        return len(self.img_df)

    def __getitem__(self, idx):
        
        # Extracting all the other columns except label_name
        img_cols = [ i for i in self.img_df.columns if i not in self.label_name]
        
        image = self.img_df.iloc[[idx]][img_cols].values

        # Reshaping the array from 1*784 to 28*28
        image = image.reshape(28,28)
        # image = image.astype(float)

        # Scaling the image so that the values only range between 0 and 1
        image = image/255.0
        
        
        if self.transform:
            image = self.transform(image)
    
        image = image.to(torch.float)    
        
        if self.label_name in self.img_df.columns:
            
            if self.target_transform:
                label = self.target_transform(label)
            label = int(self.img_df.iloc[[idx]][self.label_name].values)
            return image, label
        
        # Exceptions for test where labels are absent
        else :
            return image

In [ ]:
## Illustration of creating a validation set 

from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))

train_indices , test_indices = train_test_split(indices, test_size=0.1, stratify=train_df['label'])
# train_indices , test_indices = train_test_split(indices, test_size=0.1)

len(train_indices) , len(test_indices) , len(train_df)

train_subset = train_df.loc[train_indices]
val_subset = train_df.loc[test_indices]

print("Distribution of target values in training dataset ; ")
print( train_subset['label'].value_counts().sort_index() / train_subset['label'].value_counts().sort_index().sum() )

print("Distribution of target values in validation dataset ; ")
print( val_subset['label'].value_counts().sort_index() / val_subset['label'].value_counts().sort_index().sum() )

In [ ]:
from torchvision import transforms
import os

# Crerating a temp dataset
train_csv_name = "fashion-mnist_train.csv"
test_csv_name = "fashion-mnist_test.csv"
img_dir = ""

# Converting X variables to Tensors
transforms = transforms.Compose( [transforms.ToTensor() , transforms.Normalize((0.5,), (0.5,)) , ] )

label_name = "label"

train_dataset = CustomMNISTDataset(csv_name = train_csv_name , img_dir = img_dir , transform = transforms , target_transform = None , label_name = label_name)

# Inspecting the fist line item under dataset
x0 , y0 = train_dataset[0]
print(x0.shape , y0)

In [ ]:
# Ploting some of the datapoints in the dataset
import matplotlib.pyplot as plt

# sample_img , sample_lbl = temp_train_dataset[3]
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
figure.add_subplot(rows, cols, 1)
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
    sample_img , sample_lbl = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(sample_lbl)
    plt.axis("off")
    plt.imshow(sample_img.squeeze(), cmap="gray")
plt.show()

In [ ]:
## Checking if the GPU is being used properly . 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.cuda.is_available()
x0 = x0.to(device)
print("x0" , x0.is_cuda)

In [ ]:
from torch.utils.data import SubsetRandomSampler
from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))
train_indices , valid_indices = train_test_split(indices, test_size=0.95, stratify=train_df['label'])



In [ ]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(test_indices)

train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=64, sampler=train_sampler, num_workers=16)
valid_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=valid_sampler, num_workers=16)


In [ ]:
from torch import nn

# Get cpu or gpu device for training.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class Multiply(nn.Module):
    def __init__(self, factor):
        super(Multiply, self).__init__()
        self.factor = factor

    def forward(self, x):
        return x*self.factor

class MyOwnNeuralNetwork(nn.Module):
    def __init__(self,m,alpha):
        super(MyOwnNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.width = m
        self.alpha = alpha
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, self.width),
            nn.Linear(self.width, self.width),
            nn.LeakyReLU(self.alpha),
            Multiply(1./np.sqrt(1+alpha**2)), 
            nn.Linear(self.width, self.width),
            nn.LeakyReLU(self.alpha),
            Multiply(1./np.sqrt(1+alpha**2)), 
            nn.Linear(self.width, 10)
            ## Softmax layer ignored since the loss function defined is nn.CrossEntropy()
        )
        nn.init.constant_(self.linear_relu_stack[0].bias, 0)
        nn.init.normal_(self.linear_relu_stack[0].weight, mean=0, std=np.sqrt(1./m))
        nn.init.constant_(self.linear_relu_stack[1].bias, 0)
        nn.init.normal_(self.linear_relu_stack[1].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(self.linear_relu_stack[4].bias, 0)
        nn.init.normal_(self.linear_relu_stack[4].weight, mean=0, std=np.sqrt(2./m))
        nn.init.normal_(self.linear_relu_stack[7].weight, mean=0, std=np.sqrt(1))
        nn.init.constant_(self.linear_relu_stack[7].bias, 0)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return  logits
    
    


In [ ]:
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
def train(dataloader, model, loss_fn, optimizer):
    
    # Total size of dataset for reference
    size = 0
    
    # places your model into training mode
    model.train()
    
    # loss batch
    batch_loss = {}
    batch_accuracy = {}
    
    correct = 0
    _correct = 0
    
    
    
    # Gives X , y for each batch
    for batch, (X, y) in enumerate(dataloader):
        
        # Converting device to cuda
        X, y = X.to(device), y.to(device)
        model.to(device)
        
        # Compute prediction error / loss
        # 1. Compute y_pred 
        # 2. Compute loss between y and y_pred using selectd loss function
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation on optimizing for loss
        # 1. Sets gradients as 0 
        # 2. Compute the gradients using back_prop
        # 3. update the parameters using the gradients from step 2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _correct = (y_pred.argmax(1) == y).type(torch.float).sum().item()
        _batch_size = len(X)
        
        correct += _correct
        
        # Updating loss_batch and batch_accuracy
        batch_loss[batch] = loss.item()
        batch_accuracy[batch] = _correct/_batch_size
        
        size += _batch_size
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}]")
    
    correct/=size
    print(f"Train Accuracy: {(100*correct):>0.1f}%")
    
    return batch_loss , batch_accuracy

In [ ]:
def validation(dataloader, model, loss_fn):
    
    # Total size of dataset for reference
    size = 0
    num_batches = len(dataloader)
    
    # Setting the model under evaluation mode.
    model.eval()

    test_loss, correct = 0, 0
    
    _correct = 0
    _batch_size = 0
    
    batch_loss = {}
    batch_accuracy = {}
    
    with torch.no_grad():
        
        # Gives X , y for each batch
        for batch , (X, y) in enumerate(dataloader):
            
            X, y = X.to(device), y.to(device)
            model.to(device)
            pred = model(X)
            
            batch_loss[batch] = loss_fn(pred, y).item()
            test_loss += batch_loss[batch]
            _batch_size = len(X)
            
            _correct = (pred.argmax(1) == y).type(torch.float).sum().item()
            correct += _correct
            
            size+=_batch_size
            batch_accuracy[batch] = _correct/_batch_size
            
            
            
    
    ## Calculating loss based on loss function defined
    test_loss /= num_batches
    
    ## Calculating Accuracy based on how many y match with y_pred
    correct /= size
    
    print(f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return batch_loss , batch_accuracy

In [ ]:
train_batch_loss_all = []
train_batch_accuracy_all = []
valid_batch_accuracy_all = []
valid_batch_loss_all = []
train_epoch_no_all = []
valid_epoch_no_all = []
train_loss_all = []
valid_loss_all = []

epochs = 300
m = 2000
alphalist = [-2.0, -1, 0.05, 0, 0.01]
import pickle
for k in range(len(alphalist)):
    train_batch_loss = []
    train_batch_accuracy = []
    valid_batch_accuracy = []
    valid_batch_loss = []
    train_epoch_no = []
    valid_epoch_no = []
    train_loss = []
    valid_loss = []
    alpha = alphalist[k]
    model = MyOwnNeuralNetwork(m,alpha).to(device)
    ## Defining optimizer and loss functions 
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0)
    print(model)
    print(f"Alpha {alpha}\n-------------------------------")
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        _train_batch_loss , _train_batch_accuracy = train(train_dataloader, model, loss_fn, optimizer)
        _valid_batch_loss , _valid_batch_accuracy = validation(valid_dataloader, model, loss_fn)
        training_losses = []
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            model.to(device)
            pred = model(X)
            batch_loss = loss_fn(pred, y).item()
            training_losses.append(batch_loss) 
        train_loss.append(np.mean(training_losses))
        valid_losses = []
        for batch, (X, y) in enumerate(valid_dataloader):
            X, y = X.to(device), y.to(device)
            model.to(device)
            pred = model(X)
            batch_loss = loss_fn(pred, y).item()
            valid_losses.append(batch_loss) 
        valid_loss.append(np.mean(valid_losses))
        for i in range(len(_train_batch_loss)):
            train_batch_loss.append(_train_batch_loss[i])
            train_batch_accuracy.append(_train_batch_accuracy[i])
            train_epoch_no.append( t + float((i+1)/len(_train_batch_loss)))     
        for i in range(len(_valid_batch_loss)):
            valid_batch_loss.append(_valid_batch_loss[i])
            valid_batch_accuracy.append(_valid_batch_accuracy[i])
            valid_epoch_no.append( t + float((i+1)/len(_valid_batch_loss)))   
    train_batch_loss_all.append(train_batch_loss)
    train_batch_accuracy_all.append(train_batch_accuracy)
    train_epoch_no_all.append(train_epoch_no)
    valid_batch_loss_all.append(valid_batch_loss)
    valid_batch_accuracy_all.append(valid_batch_accuracy)
    valid_epoch_no_all.append(valid_epoch_no)
    train_loss_all.append(train_loss)
    valid_loss_all.append(valid_loss)
    with open('fmnist_train_valid'+str(filenum).zfill(3)+'_'+str(expid).zfill(3)+'.pickle', 'wb') as handle:
        pickle.dump({'train_epoch_no_all':train_epoch_no_all, \
                     'train_batch_loss_all':train_batch_loss_all, \
                    'valid_epoch_no_all':valid_epoch_no_all, \
                     'valid_batch_loss_all':valid_batch_loss_all, \
                     'valid_loss_all':valid_loss_all, \
                     'train_loss_all':train_loss_all, \
                     'valid_batch_accuracy_all' : valid_batch_accuracy_all, \
                     'train_batch_accuracy_all' : train_batch_accuracy_all, \
                    }, handle)
    print("Done!")
